In [4]:
import os
from dotenv import load_dotenv
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

In [5]:
load_dotenv('sql_credentials.env')

db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')
db_host = os.getenv('DB_HOST')
db_name = os.getenv('DB_NAME')

db_connection_str = f'mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}'
engine = create_engine(db_connection_str)


### Objectives:

- Gather all necesary data for modeling and EDA


#### Step 1

Here I'll query for all rows in results table, I'll fetch all foreign keys related to tables we'll need to get data from, plus the grid position. I'll join that with the race table to get other foreign keys plus the year, round, and date of the race. 

In [6]:
results_query = """
SELECT r.resultId, r.raceId, r.driverId, r.constructorId, r.grid, r.position, races.year, races.round, races.circuitId, races.date
FROM results r
JOIN races ON r.raceId = races.raceId
"""
results_df = pd.read_sql(results_query, con=engine)
results_df.info()
results_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26699 entries, 0 to 26698
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   resultId       26699 non-null  int64  
 1   raceId         26699 non-null  int64  
 2   driverId       26699 non-null  int64  
 3   constructorId  26699 non-null  int64  
 4   grid           26699 non-null  int64  
 5   position       15756 non-null  float64
 6   year           26699 non-null  int64  
 7   round          26699 non-null  int64  
 8   circuitId      26699 non-null  int64  
 9   date           26699 non-null  object 
dtypes: float64(1), int64(8), object(1)
memory usage: 2.0+ MB


,resultId,raceId,driverId,constructorId,grid,position,year,round,circuitId,date
0,1,18,1,1,1,1.0,2008,1,1,2008-03-16
1,2,18,2,2,5,2.0,2008,1,1,2008-03-16
2,3,18,3,3,7,3.0,2008,1,1,2008-03-16
3,4,18,4,4,11,4.0,2008,1,1,2008-03-16
4,5,18,5,1,3,5.0,2008,1,1,2008-03-16



#### Step 2

Here I'll calculate the driver age at the time of the race, for that I'll get the driver's DOB and compare that to each entry's date to get the drivers age at that time.

In [7]:
driver_age_query = """
SELECT d.driverId, d.dob
FROM drivers d
"""
drivers_df = pd.read_sql(driver_age_query, con=engine)

results_df = results_df.merge(drivers_df, on="driverId", how="left")
results_df['driver_age'] = results_df.apply(
    lambda row: row['date'].year - row['dob'].year - 
                ((row['date'].month, row['date'].day) < (row['dob'].month, row['dob'].day)),
    axis=1
)

results_df.head()

,resultId,raceId,driverId,constructorId,grid,position,year,round,circuitId,date,dob,driver_age
0,1,18,1,1,1,1.0,2008,1,1,2008-03-16,1985-01-07,23
1,2,18,2,2,5,2.0,2008,1,1,2008-03-16,1977-05-10,30
2,3,18,3,3,7,3.0,2008,1,1,2008-03-16,1985-06-27,22
3,4,18,4,4,11,4.0,2008,1,1,2008-03-16,1981-07-29,26
4,5,18,5,1,3,5.0,2008,1,1,2008-03-16,1981-10-19,26



#### Step 3

Here I'll calculate the driver's experience (Number of GPs entered). For that I'll start by sorting the current `results_df` by `date` and `resultId`. Then I'll create a cumulative count of appearance for each driver.

In [8]:
results_df = results_df.sort_values(by=['date', 'resultId']).reset_index(drop=True)

results_df['driver_experience'] = results_df.groupby('driverId').cumcount()

results_df.tail()

,resultId,raceId,driverId,constructorId,grid,position,year,round,circuitId,date,dob,driver_age,driver_experience
26694,26700,1141,832,6,0,NaN,2024,21,18,2024-11-03,1994-09-01,30,204
26695,26701,1141,861,3,16,NaN,2024,21,18,2024-11-03,2003-05-27,21,5
26696,26702,1141,848,3,7,NaN,2024,21,18,2024-11-03,1996-03-23,28,101
26697,26703,1141,840,117,10,NaN,2024,21,18,2024-11-03,1998-10-29,26,164
26698,26704,1141,807,210,17,NaN,2024,21,18,2024-11-03,1987-08-19,37,226


We can see that the 2nd row, which corresponds to Franco Colapinto (driverId=861) has 6 races which is correct after the 2024 Brazilian GP


#### Step 4

Similar to last step, I'll calculate the driver's experience with it's current team. For that I'll create a cumulative count of appearances for each `driverId` and `constructorId` combination

In [9]:
results_df['driver_constructor_experience'] = results_df.groupby(['driverId', 'constructorId']).cumcount()

results_df.tail()

,resultId,raceId,driverId,constructorId,grid,position,year,round,circuitId,date,dob,driver_age,driver_experience,driver_constructor_experience
26694,26700,1141,832,6,0,NaN,2024,21,18,2024-11-03,1994-09-01,30,204,85
26695,26701,1141,861,3,16,NaN,2024,21,18,2024-11-03,2003-05-27,21,5,5
26696,26702,1141,848,3,7,NaN,2024,21,18,2024-11-03,1996-03-23,28,101,63
26697,26703,1141,840,117,10,NaN,2024,21,18,2024-11-03,1998-10-29,26,164,86
26698,26704,1141,807,210,17,NaN,2024,21,18,2024-11-03,1987-08-19,37,226,42


#### Step 5

Now I want to calculate both the driver's all time wins and the driver's all time wins with that specific constructor. Using a temporary `win_indicator`, and similary to last steps, using a cumulative count, then dropping the temporary column as it's no longer needed.

In [10]:
results_df['win_indicator'] = results_df['position'] == 1.0

results_df['driver_wins'] = results_df.groupby('driverId')['win_indicator'].cumsum()
results_df['constructor_wins'] = results_df.groupby(['driverId', 'constructorId'])['win_indicator'].cumsum()

results_df.drop(columns=['win_indicator'], inplace=True)


#### Step 6

For this step I need to make a new query: I need `driver_points` and `driver_standings` after each race, that I'll then merge to `results_df`

In [11]:
driver_standings_query = """
SELECT ds.raceId, ds.driverId, ds.points AS driver_points, ds.position AS driver_standing
FROM driverStandings ds
"""
driver_standings_df = pd.read_sql(driver_standings_query, con=engine)
results_df = results_df.merge(driver_standings_df, on=["raceId", "driverId"], how="left")

results_df.head()

,resultId,raceId,driverId,constructorId,grid,position,year,round,circuitId,date,dob,driver_age,driver_experience,driver_constructor_experience,driver_wins,constructor_wins,driver_points,driver_standing
0,20025,833,642,51,1,1.0,1950,1,9,1950-05-13,1906-10-30,43,0,0,1,1,9.0,1.0
1,20026,833,786,51,2,2.0,1950,1,9,1950-05-13,1898-06-09,51,0,0,0,0,6.0,2.0
2,20027,833,686,51,4,3.0,1950,1,9,1950-05-13,1911-07-02,38,0,0,0,0,4.0,3.0
3,20028,833,704,154,6,4.0,1950,1,9,1950-05-13,1904-10-08,45,0,0,0,0,3.0,4.0
4,20029,833,627,154,9,5.0,1950,1,9,1950-05-13,1905-11-05,44,0,0,0,0,2.0,5.0



#### Step 7

Same as last step, but this time for constructor standings data

In [12]:
constructor_standings_query = """
SELECT cs.raceId, cs.constructorId, cs.points AS constructor_points, cs.position AS constructor_standing
FROM constructorStandings cs
"""
constructor_standings_df = pd.read_sql(constructor_standings_query, con=engine)
results_df = results_df.merge(constructor_standings_df, on=["raceId", "constructorId"], how="left")

results_df.head()

,resultId,raceId,driverId,constructorId,grid,position,year,round,circuitId,date,dob,driver_age,driver_experience,driver_constructor_experience,driver_wins,constructor_wins,driver_points,driver_standing,constructor_points,constructor_standing
0,20025,833,642,51,1,1.0,1950,1,9,1950-05-13,1906-10-30,43,0,0,1,1,9.0,1.0,NaN,NaN
1,20026,833,786,51,2,2.0,1950,1,9,1950-05-13,1898-06-09,51,0,0,0,0,6.0,2.0,NaN,NaN
2,20027,833,686,51,4,3.0,1950,1,9,1950-05-13,1911-07-02,38,0,0,0,0,4.0,3.0,NaN,NaN
3,20028,833,704,154,6,4.0,1950,1,9,1950-05-13,1904-10-08,45,0,0,0,0,3.0,4.0,NaN,NaN
4,20029,833,627,154,9,5.0,1950,1,9,1950-05-13,1905-11-05,44,0,0,0,0,2.0,5.0,NaN,NaN


#### Warning

Now we have a problem here:
As we can see the standings and data we got are for ***after*** each race, and since we want to predict a race result (finishing position), we need to have the standings from ***before*** the race. 
This means we will have to do some logic:
1. For the first race ever (1950 round 1), `driver_wins`, `constructor_wins`, `driver_points`, `driver_standing`, `constructor_points` and `constructor_standing` will be set to zero.
2. For the first race of every season, `driver_points`, `driver_standing`, `constructor_points` and `constructor_standing` will be 0, but `driver_wins` and `constructor_wins` will be carried over from the past race.
3. All data we have now on an entry, will be moved 1 race ahead. I.e. 2022 round 4 has the data for *after* that race, that data will be the *starting* data for the next race. So 2022 round 4 data will now be 2022 round 5 data.
4. Keeping the last step logic, if a season like 2022 has 22 rounds, we will end up with a round 23. So we need to check if the data we're handling is from the last round of the season, therefore we won't append that new entry to the new DataFrame. 


#### Step 8

Shift data. As this is a more complicated step, I'll go step by step in the code block

In [13]:
# Columns that reset to zero at the start of each season
stats_to_reset = ['driver_points', 'driver_standing', 'constructor_points', 'constructor_standing']
# Columns that carry over across seasons except for the first race of the entire dataset
stats_to_carry = ['driver_wins', 'constructor_wins']

# Define the columns to keep in the shifted DataFrame
columns_to_keep = ['year', 'round', 'driverId'] + stats_to_reset + stats_to_carry

# Initialize the shifted DataFrame with only the necessary columns
results_df_shifted = pd.DataFrame(columns=columns_to_keep)

# Track if it's the first race in the entire DataFrame
is_first_race_ever = True

# Calculate the maximum round for each year
max_rounds_per_year = results_df.groupby('year')['round'].max()

# Initialize a variable to hold stats_to_carry values for carryover between seasons
prev_season_carryover = None

# Iterate through rows in the original DataFrame, but only with necessary columns
for i, row in results_df[columns_to_keep].iterrows():
    # Create a new row with the incremented round
    new_row = row.copy()
    new_row['round'] += 1  # Increment round by 1 to shift forward

    # Check if this is the first race of the season
    if row['round'] == 1:
        # Create a zeroed row for the start of the season
        zeroed_row = row.copy()

        # Set stats_to_reset to 0 for the start of each season
        zeroed_row[stats_to_reset] = 0

        # Set stats_to_carry to 0 only if this is the first race of the entire dataset
        if is_first_race_ever:
            zeroed_row[stats_to_carry] = 0
            is_first_race_ever = False
        else:
            # If not the first race ever, let stats_to_carry keep the previous year's value
            zeroed_row[stats_to_carry] = prev_season_carryover

        # Append the zeroed row to the shifted DataFrame
        results_df_shifted = pd.concat([results_df_shifted, zeroed_row.to_frame().T], ignore_index=True)

    # Check if this row is the last round of the season
    is_last_round_of_season = row['round'] == max_rounds_per_year[row['year']]

    if is_last_round_of_season:
        # Skip adding the last round's shifted row for the season end and add this row data to carry it for the first round of next season
        prev_season_carryover = row[stats_to_carry]
        continue
    else: 
        # Append the shifted row to the new DataFrame
        results_df_shifted = pd.concat([results_df_shifted, new_row.to_frame().T], ignore_index=True)

# After constructing the DataFrame, convert columns to int so we keep the same format as results_df
results_df_shifted['year'] = results_df_shifted['year'].astype('int64')
results_df_shifted['round'] = results_df_shifted['round'].astype('int64')
results_df_shifted['driverId'] = results_df_shifted['driverId'].astype('int64')

# Display the resulting DataFrame to verify
results_df_shifted.tail(10)

/var/folders/4z/6wzfzsy94r16_shjk6y6ndmc0000gn/T/ipykernel_23151/3076887936.py:44: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df_shifted = pd.concat([results_df_shifted, zeroed_row.to_frame().T], ignore_index=True)


,year,round,driverId,driver_points,driver_standing,constructor_points,constructor_standing,driver_wins,constructor_wins
26643,2024,21,840,24.0,11.0,86.0,5.0,0.0,0.0
26644,2024,21,861,5.0,18.0,17.0,8.0,0.0,0.0
26645,2024,21,839,5.0,19.0,14.0,9.0,1.0,1.0
26646,2024,21,822,0.0,23.0,0.0,10.0,10.0,0.0
26647,2024,21,855,0.0,21.0,0.0,10.0,0.0,0.0
26648,2024,21,859,2.0,20.0,36.0,7.0,0.0,0.0
26649,2024,21,815,150.0,8.0,512.0,3.0,6.0,5.0
26650,2024,21,4,62.0,9.0,86.0,5.0,32.0,0.0
26651,2024,21,848,12.0,14.0,17.0,8.0,0.0,0.0
26652,2024,21,852,22.0,12.0,36.0,7.0,0.0,0.0



#### Step 9

Merge `results_df` with `results_df_shifted` into a new df: `merged_df`

In [14]:
columns_to_shift = stats_to_reset + stats_to_carry
merged_df = results_df.drop(columns=columns_to_shift).merge(
    results_df_shifted,
    on=['year', 'round', 'driverId'],
    how='left'
)

merged_df = merged_df.rename(columns={f'{col}': col for col in columns_to_shift})

merged_df.tail(10)

,resultId,raceId,driverId,constructorId,grid,position,year,round,circuitId,date,dob,driver_age,driver_experience,driver_constructor_experience,driver_points,driver_standing,constructor_points,constructor_standing,driver_wins,constructor_wins
26776,26695,1141,815,9,12,11.0,2024,21,18,2024-11-03,1990-01-26,34,279,86,150.0,8.0,512.0,3.0,6.0,5.0
26777,26696,1141,860,210,15,12.0,2024,21,18,2024-11-03,2005-05-08,19,2,1,NaN,NaN,NaN,NaN,NaN,NaN
26778,26697,1141,822,15,11,13.0,2024,21,18,2024-11-03,1989-08-28,35,243,20,0.0,23.0,0.0,10.0,10.0,0.0
26779,26698,1141,4,117,9,14.0,2024,21,18,2024-11-03,1981-07-29,43,400,42,62.0,9.0,86.0,5.0,32.0,0.0
26780,26699,1141,855,15,19,15.0,2024,21,18,2024-11-03,1999-05-30,25,64,20,0.0,21.0,0.0,10.0,0.0,0.0
26781,26700,1141,832,6,0,NaN,2024,21,18,2024-11-03,1994-09-01,30,204,85,240.0,5.0,537.0,2.0,4.0,4.0
26782,26701,1141,861,3,16,NaN,2024,21,18,2024-11-03,2003-05-27,21,5,5,5.0,18.0,17.0,8.0,0.0,0.0
26783,26702,1141,848,3,7,NaN,2024,21,18,2024-11-03,1996-03-23,28,101,63,12.0,14.0,17.0,8.0,0.0,0.0
26784,26703,1141,840,117,10,NaN,2024,21,18,2024-11-03,1998-10-29,26,164,86,24.0,11.0,86.0,5.0,0.0,0.0
26785,26704,1141,807,210,17,NaN,2024,21,18,2024-11-03,1987-08-19,37,226,42,31.0,10.0,46.0,6.0,0.0,0.0


#### Step 10

Filter `merged_df` to keep only rows where `year` is 2010 or later. Why I'm doing this? Because 2010 is the last time F1 made a big change in the points award system. So for simplicity, instead of converting all the previous races for the current point system, I'll work with all the entries from 2010 or later.


In [15]:
merged_df = merged_df[merged_df['year'] >= 2010].reset_index(drop=True)
merged_df.head()

,resultId,raceId,driverId,constructorId,grid,position,year,round,circuitId,date,dob,driver_age,driver_experience,driver_constructor_experience,driver_points,driver_standing,constructor_points,constructor_standing,driver_wins,constructor_wins
0,20323,337,4,6,3,1.0,2010,1,3,2010-03-14,1981-07-29,28,140,0,0.0,0.0,0.0,0.0,0.0,0.0
1,20324,337,13,6,2,2.0,2010,1,3,2010-03-14,1981-04-25,28,116,63,0.0,0.0,0.0,0.0,0.0,0.0
2,20325,337,1,1,4,3.0,2010,1,3,2010-03-14,1985-01-07,25,52,52,0.0,0.0,0.0,0.0,0.0,0.0
3,20326,337,20,9,1,4.0,2010,1,3,2010-03-14,1987-07-03,22,43,17,0.0,0.0,0.0,0.0,0.0,0.0
4,20327,337,3,131,5,5.0,2010,1,3,2010-03-14,1985-06-27,24,70,0,0.0,0.0,0.0,0.0,0.0,0.0


#### Step 11

Calculate Circuit Danger Metric. What is this? One of the tables is *status* which displays the status for each *results* entry. And since each of those entries corresponds to one *race*, we can calculate how many incidents there were on each circuit, and the total of races on that circuit. So `circuit_danger` will result of dividing the total of incidents on a circuit by the total races on that circuit, from 2010 or later.

In [16]:
driver_experience_query = """
SELECT 
    c.circuitId,
    c.name,
    COUNT(*) AS count,
    total_races.total,
    COUNT(*) * 1.0 / total_races.total AS circuit_danger
FROM 
    races r
JOIN 
    results res ON r.raceId = res.raceId
JOIN 
    circuits c ON r.circuitId = c.circuitId
JOIN 
    (SELECT circuitId, COUNT(*) AS total 
     FROM races 
     WHERE year >= 2010 
     GROUP BY circuitId) AS total_races
    ON r.circuitId = total_races.circuitId
WHERE 
    res.statusId IN (3, 4) 
    AND r.year >= 2010
GROUP BY 
    c.circuitId, c.name, total_races.total
ORDER BY 
    circuit_danger DESC;
"""
circuit_df = pd.read_sql(driver_experience_query, con=engine)

circuit_df

,circuitId,name,count,total,circuit_danger
0,76,Autodromo Internazionale del Mugello,6,1,6.00000
1,35,Korean International Circuit,13,4,3.25000
2,6,Circuit de Monaco,36,14,2.57143
3,15,Marina Bay Street Circuit,28,13,2.15385
4,1,Albert Park Grand Prix Circuit,24,13,1.84615
5,71,Sochi Autodrom,14,8,1.75000
6,7,Circuit Gilles Villeneuve,22,13,1.69231
7,12,Valencia Street Circuit,5,3,1.66667
8,68,Buddh International Circuit,5,3,1.66667
9,18,Autódromo José Carlos Pace,23,14,1.64286


#### Step 12

Here I'll merge `circuit_danger` to the `merged_df`. The rest of the values will be used for EDA.

In [17]:
circuit_danger_df = circuit_df[['circuitId', 'circuit_danger']]

merged_df = merged_df.merge(circuit_danger_df, on='circuitId', how='left')

merged_df.tail()

,resultId,raceId,driverId,constructorId,grid,position,year,round,circuitId,date,...,driver_age,driver_experience,driver_constructor_experience,driver_points,driver_standing,constructor_points,constructor_standing,driver_wins,constructor_wins,circuit_danger
6371,26700,1141,832,6,0,NaN,2024,21,18,2024-11-03,...,30,204,85,240.0,5.0,537.0,2.0,4.0,4.0,1.64286
6372,26701,1141,861,3,16,NaN,2024,21,18,2024-11-03,...,21,5,5,5.0,18.0,17.0,8.0,0.0,0.0,1.64286
6373,26702,1141,848,3,7,NaN,2024,21,18,2024-11-03,...,28,101,63,12.0,14.0,17.0,8.0,0.0,0.0,1.64286
6374,26703,1141,840,117,10,NaN,2024,21,18,2024-11-03,...,26,164,86,24.0,11.0,86.0,5.0,0.0,0.0,1.64286
6375,26704,1141,807,210,17,NaN,2024,21,18,2024-11-03,...,37,226,42,31.0,10.0,46.0,6.0,0.0,0.0,1.64286


#### Step 13

Drop Nulls

In [18]:
merged_df = merged_df.dropna()
merged_df.head()

,resultId,raceId,driverId,constructorId,grid,position,year,round,circuitId,date,...,driver_age,driver_experience,driver_constructor_experience,driver_points,driver_standing,constructor_points,constructor_standing,driver_wins,constructor_wins,circuit_danger
0,20323,337,4,6,3,1.0,2010,1,3,2010-03-14,...,28,140,0,0.0,0.0,0.0,0.0,0.0,0.0,0.66667
1,20324,337,13,6,2,2.0,2010,1,3,2010-03-14,...,28,116,63,0.0,0.0,0.0,0.0,0.0,0.0,0.66667
2,20325,337,1,1,4,3.0,2010,1,3,2010-03-14,...,25,52,52,0.0,0.0,0.0,0.0,0.0,0.0,0.66667
3,20326,337,20,9,1,4.0,2010,1,3,2010-03-14,...,22,43,17,0.0,0.0,0.0,0.0,0.0,0.0,0.66667
4,20327,337,3,131,5,5.0,2010,1,3,2010-03-14,...,24,70,0,0.0,0.0,0.0,0.0,0.0,0.0,0.66667


#### Step 14

Drop unnecesary columns

In [20]:
merged_df = merged_df.drop(columns=['date', 'dob', 'resultId', 'raceId'])
merged_df.head()

,driverId,constructorId,grid,position,year,round,circuitId,driver_age,driver_experience,driver_constructor_experience,driver_points,driver_standing,constructor_points,constructor_standing,driver_wins,constructor_wins,circuit_danger
0,4,6,3,1.0,2010,1,3,28,140,0,0.0,0.0,0.0,0.0,0.0,0.0,0.66667
1,13,6,2,2.0,2010,1,3,28,116,63,0.0,0.0,0.0,0.0,0.0,0.0,0.66667
2,1,1,4,3.0,2010,1,3,25,52,52,0.0,0.0,0.0,0.0,0.0,0.0,0.66667
3,20,9,1,4.0,2010,1,3,22,43,17,0.0,0.0,0.0,0.0,0.0,0.0,0.66667
4,3,131,5,5.0,2010,1,3,24,70,0,0.0,0.0,0.0,0.0,0.0,0.0,0.66667


#### Step 15

Get Dummies for `driverId`, `circuitId` and `constructorId`

In [21]:
merged_df = pd.get_dummies(merged_df, columns=['driverId', 'circuitId', 'constructorId'])
merged_df.head()

,grid,position,year,round,driver_age,driver_experience,driver_constructor_experience,driver_points,driver_standing,constructor_points,...,constructorId_205,constructorId_206,constructorId_207,constructorId_208,constructorId_209,constructorId_210,constructorId_211,constructorId_213,constructorId_214,constructorId_215
0,3,1.0,2010,1,28,140,0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False
1,2,2.0,2010,1,28,116,63,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False
2,4,3.0,2010,1,25,52,52,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False
3,1,4.0,2010,1,22,43,17,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False
4,5,5.0,2010,1,24,70,0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False


#### Step 16

Export the data frame as `final_df.csv`

In [23]:
merged_df.to_csv('final_df.csv', index=False)